In [80]:
# housekeeping: extract list of 2024B targets from xml file, since no easy list was left

import xml.etree.ElementTree as ET

tree = ET.parse("../../Proposals/Gemini/Gemini 2024B/Gemini_QEB.speckle_iq70.xml")
root = tree.getroot()
targets = set()
for target in root.findall("targets/sidereal"):
    name = target.find("name").text
    targets.add(name)

with open("../../Proposals/Gemini/Gemini 2024B/Gemini 2024B targets.csv", "w") as f:
    f.write("Gemini 2024B Proposal\n")
    f.write("All targets included in Gemini 2024B Proposal\n")
    for target in targets:
        f.write(f"{target}\n")

len(targets)

136

In [81]:
# use the target list creator to make a file that can be imported into the PIT
# we decided to use the same target list as the 2024B proposal

from functools import partial
from sqlite3 import connect
import sys

from astropy.table import Table

sys.path.append("../../")
import astropaul.targetlistcreator as tlc
import astropaul.gemini as gemini

conn = connect("../../django/TargetDB/db.sqlite3")

creator = tlc.TargetListCreator(connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="tess"),
    tlc.add_lists,
    tlc.add_speckle,
    gemini.add_gemini_speckle_params,
]

targets = Table.from_pandas(creator.calculate())
gemini_targets = targets[targets["list_Gemini_2024B_Proposal"]]
gemini_targets.rename_column("target_name", "Name")
gemini_targets.rename_column("ra", "RAJ2000")
gemini_targets.rename_column("dec", "DecJ2000")
gemini_targets.rename_column("pmra", "pmRA")
gemini_targets.rename_column("pmdec", "pmDec")
gemini_targets.rename_column("Vmag", "V")
gemini_targets.rename_column("gemini_iq70_speckle_program_time", "IQ70 Program Time")
gemini_targets.rename_column("gemini_iq85_speckle_program_time", "IQ85 Program Time")

# # break list into north & south and write to file
# cutpoint = 25
# lists = {
#     "North": gemini_targets[gemini_targets["DecJ2000"] > -cutpoint],
#     "South": gemini_targets[gemini_targets["DecJ2000"] < cutpoint],
# }
# for name, list in lists.items():
#     list[["Name", "RAJ2000", "DecJ2000", "pmRA", "pmDec", "V"]].write(f"Gemini 2025A Proposal {name}.csv", overwrite=True)

gemini_targets


Name,target_source,target_type,RAJ2000,DecJ2000,pmRA,pmDec,parallax,V,Teff,list_Featured_targets,list_Gemini_2024B_Proposal,list_HQND,list_Kostov_2022,list_Kostov_2023,list_LBT_2023-12-06,list_LBT_2023B,list_Speckle_Resolved,speckle_count,gemini_iq70_speckle_sequences,gemini_iq85_speckle_sequences,gemini_iq70_speckle_exposure,gemini_iq85_speckle_exposure,IQ70 Program Time,IQ85 Program Time
str14,str28,str6,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,int64,int64,int64,float64,float64,float64,float64
TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.9253097979945,29.1345840372292,1.09215,-1.99413,0.347484,15.116,6445.0,False,True,False,False,True,False,False,False,0,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 58539786,Kostov 2023 arXiv:2309.14200,QuadEB,65.8340211217007,29.1427494807598,5.51137,-5.3655,0.83413,15.266,5597.0,False,True,False,False,True,False,False,False,0,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 219006972,Kostov 2023 arXiv:2309.14200,QuadEB,221.202871081398,66.3786726710139,6.64058,-3.44764,0.703547,14.312,5878.0,False,True,False,False,True,False,False,False,0,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 240256832,Kostov 2023 arXiv:2309.14200,QuadEB,322.296886956971,44.0333268274813,-16.4191,-17.5365,--,13.242,--,False,True,False,False,True,False,False,False,0,7,14,448.0,896.0,886.882656,1418.0637120000001
TIC 120911334,Kostov 2023 arXiv:2309.14200,QuadEB,73.8621033608711,38.3129564434802,-4.61153,0.315877,--,15.2,--,False,True,False,False,True,False,False,False,0,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 430745185,Kostov 2023 arXiv:2309.14200,QuadEB,333.69028679781,49.2559110614137,-8.45255,-3.75994,0.561646,13.204,6282.0,False,True,False,False,True,False,False,False,0,7,14,448.0,896.0,886.882656,1418.0637120000001
TIC 371583423,Kostov 2023 arXiv:2309.14200,QuadEB,343.115417912764,55.4330449713393,-6.12292,-8.73148,--,13.881,--,False,True,False,False,True,False,False,False,0,7,14,448.0,896.0,886.882656,1418.0637120000001
TIC 414026507,Kostov 2022 arXiv:2202.05790,QuadEB,336.837716720964,56.7403620051295,-3.66449,-2.95647,0.242427,10.899,9187.0,False,True,True,True,False,True,True,False,5,5,10,320.0,640.0,735.1166400000001,1114.53168
TIC 121511673,Kostov 2023 arXiv:2309.14200,QuadEB,74.9719483022246,40.4020715331373,0.987281,-2.48414,0.306497,14.208,9336.0,False,True,False,False,True,False,False,False,0,11,22,704.0,1408.0,1190.414688,2025.127776


In [82]:
# filter out unobservable targets
import astroplan as ap
from astropy.coordinates import SkyCoord
from astropy.table import vstack
from astropy.time import Time
import astropy.units as u

constraints = [
    ap.AltitudeConstraint(30 * u.deg, 80 * u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
]

segments = [
    ("North", ap.Observer.at_site("Gemini North"), gemini_targets[gemini_targets["DecJ2000"] > 0]),
    ("South", ap.Observer.at_site("Gemini South"), gemini_targets[gemini_targets["DecJ2000"] < 0]),
]

semester = (Time("2025-01-01"), Time("2025-06-30"))

for name, observer, list in segments:
    target = ap.FixedTarget(
        coord=SkyCoord(
            frame="icrs",
            obstime=Time("2000.0", format="jyear", scale="tdb"),
            ra=list["RAJ2000"] * u.deg,
            dec=list["DecJ2000"] * u.deg,
        ),
        name=list["Name"],
    )
    list["Observable"] = ap.is_observable(constraints, observer, target, semester)
    num_targets = len(list)
    num_observable = len(list[list["Observable"]])
    print(f"{name}: {num_targets} targets, {num_observable} observable ({(num_observable / num_targets):.2f})")

proposal_targets = vstack([list[list["Observable"]] for _, _, list in segments])
proposal_targets[["Name", "RAJ2000", "DecJ2000", "pmRA", "pmDec", "V"]].write("Gemini 2025A Proposal targets.csv", overwrite=True)

print(len(gemini_targets), len(proposal_targets), len(proposal_targets) / len(gemini_targets))

North: 45 targets, 36 observable (0.80)
South: 91 targets, 62 observable (0.68)
136 98 0.7205882352941176


After going through the entire proposal process, the PIT software had problems with specific targets.  Remove them. 

In [83]:
import numpy as np

problem_targets = [
    "TIC 200094011",  # no guide star under any weather condition
    "TIC 265684107",  # PIT claimed was unobservable
    "TIC 244279814",  # PIT claimed was unobservable
    "TIC 31928452",  # PIT claimed was unobservable
    "TIC 232087348",  # PIT claimed was unobservable
    "TIC 207137124",  # PIT claimed was unobservable
    "TIC 168789840",  # PIT claimed was unobservable
    "TIC 178953404",  # PIT claimed was unobservable
    "TIC 43472931",  # PIT claimed was unobservable
    "TIC 38542448",  # PIT claimed was unobservable
    "TIC 207137124",  # Already observed in 2024B according to Jimmy
    "TIC 31928452",  # Already observed in 2024B according to Jimmy
    "TIC 168789840",  # Already observed in 2024B according to Jimmy
    "TIC 123098844",  # Already observed in 2024B according to Jimmy
    "TIC 414026507",  # Already observed in 2024B according to Jimmy
    "TIC 384735219",  # Already observed in 2024B according to Jimmy
    "TIC 384410712",  # Already observed in 2024B according to Jimmy
    "TIC 370274336",  # Already observed in 2024B according to Jimmy
    "TIC 327885074",  # Already observed in 2024B according to Jimmy
    "TIC 202664216",  # Already observed in 2024B according to Jimmy
    "TIC 45160946",  # PIT claimed prior Gemini data available for this target
    "TIC 1337279468",  # PIT claimed prior Gemini data available for this target
]

proposal_targets = proposal_targets[~np.isin(proposal_targets["Name"], problem_targets)]
proposal_targets[["Name", "RAJ2000", "DecJ2000", "pmRA", "pmDec", "V"]].write(
    "Gemini 2025A Proposal targets.csv", overwrite=True
)
len(proposal_targets)

81

* Start the PIT and open the file `Gemini 2025A Proposal Base.xml`
  * This base file has investigators, title, and other fields already populated
* Import the target list created above into the PIT
* In targets tab, sort by Dec
* Select & Copy all targets Dec < 0
* Switch to Observations tab and Paste
* Specify weather conditions and Zorro speckle (South)
* Select & Copy all targets Dec > 0
* Switch to Observations tab and Paste
* Specify weather conditions and `Alopeke speckle (North)
* Do a Save As in the PIT, to the file `Gemini 2025A Proposal.xml` (without the word base in it)
* The below code will add observing times for each target to the xml file

In [84]:
# edit the xml file saved by the PIT and add observation times
import xml.etree.ElementTree as ET

filename = "Gemini 2025A Proposal"
# filename = "north"
# filename = "south"

# first, get observation times for all targets (in hours, because that's all PIT accepts)
target_times = {
    str(name): (f"{iq70 / 3600:.2f}", f"{iq85 / 3600:.2f}")
    for name, iq70, iq85 in gemini_targets[
        ["Name", "IQ70 Program Time", "IQ85 Program Time"]
    ]
}

# next, get the mapping of the nickname used in xml file to the TIC name
tree = ET.parse(f"{filename}.xml")
root = tree.getroot()
target_map = {}
for target in root.findall("targets/sidereal"):
    designator = target.attrib["id"]
    name = target.find("name").text
    target_map[designator] = name

# finally, for each weather condition:
#   walk through each observation entry and add observing time to it for the appropriate conditions
for index, name in enumerate(["iq70", "iq85"]):
    for observation in root.findall("observations/observation"):
        target_name = target_map[observation.attrib["target"]]
        target_time = target_times[target_name][index]
        progTime = ET.Element("progTime")
        progTime.set("units", "s")
        progTime.text = target_time
        observation.append(progTime)
        time = ET.Element("time")
        time.set("units", "s")
        time.text = target_time
        observation.append(time)
    tree.write(f"{filename}_{name}.xml")


* Load new xml file into PIT
* Finish rest of proposal


In [90]:
# calculate observing time by group and by north/south/equatorial

groups = [
    (
        "Faint (V$>$13)",
        (proposal_targets["V"] >= 13)
        & (proposal_targets["list_HQND"] == False)
        & (proposal_targets["speckle_count"] > 0),
    ),
    (
        "Bright (V$<$13)",
        (proposal_targets["V"] < 13) 
        & (proposal_targets["list_HQND"] == False)
        & (proposal_targets["speckle_count"] > 0),
    ),
    ("HQUS", (proposal_targets["list_HQND"])),
    ("First Speckle", (proposal_targets["speckle_count"] == 0)),
]

regions = [
    ("North", (proposal_targets["DecJ2000"] > 25)),
    ("South", (proposal_targets["DecJ2000"] < -25)),
    ("Equatorial", (proposal_targets["DecJ2000"] > -25) & (proposal_targets["DecJ2000"] < 25)),
]

tot_count = {name: 0 for name, _ in regions}
tot_time = {name: 0 for name, _ in regions}
tot_seq = {name: 0 for name, _ in regions}
for name, group_criteria in groups:
    print(f"{name:>15s}", end=" ")
    for region, region_criteria in regions:
        group = proposal_targets[group_criteria & region_criteria]
        count = len(group)
        obs_time = np.sum(group["IQ70 Program Time"]) / 3600
        num_seq = np.sum(group["gemini_iq70_speckle_sequences"])
        print(f"& {count:2d} & {num_seq:3d} & {obs_time:5.2f}", end=" ")
        tot_count[region] += count
        tot_time[region] += obs_time
        tot_seq[region] += num_seq
    print(r"\\")
print("          TOTAL", end=" ")
for region, _ in regions:
    print(f"& {tot_count[region]:2d} & {tot_seq[region]:3d} & {tot_time[region]:5.2f}", end = " ")
print(r"\\")

time_north = np.sum(proposal_targets[proposal_targets["DecJ2000"] > 0]["IQ70 Program Time"]) / 3600
time_south = (np.sum(proposal_targets[proposal_targets["DecJ2000"] < 0]["IQ70 Program Time"]) / 3600
)
print(f"{time_north:.2f} north, {time_south:.2f} south")

 Faint (V$>$13) &  3 &  25 &  0.82 &  0 &   0 &  0.00 &  5 &  43 &  1.40 \\
Bright (V$<$13) &  0 &   0 &  0.00 &  2 &   8 &  0.37 &  0 &   0 &  0.00 \\
           HQUS &  4 &  26 &  0.94 &  1 &   7 &  0.25 &  3 &  17 &  0.65 \\
  First Speckle & 11 &  93 &  3.05 & 45 & 345 & 11.72 &  7 &  69 &  2.15 \\
          TOTAL & 18 & 144 &  4.81 & 48 & 360 & 12.33 & 15 & 129 &  4.20 \\
7.82 north, 13.53 south


* Paste the above table into the Overleaf for the experimental design
* For Phase 3, I just added the HQUS targets and their IQ85 exposure times, manually.
  * Ideally, my code could create separate Phase 1/2 and Phase 3 lists using different IQ70/85 exposure times, but I didn't have time for that this time.

In [91]:
proposal_targets[proposal_targets["list_HQND"]]

Name,target_source,target_type,RAJ2000,DecJ2000,pmRA,pmDec,parallax,V,Teff,list_Featured_targets,list_Gemini_2024B_Proposal,list_HQND,list_Kostov_2022,list_Kostov_2023,list_LBT_2023-12-06,list_LBT_2023B,list_Speckle_Resolved,speckle_count,gemini_iq70_speckle_sequences,gemini_iq85_speckle_sequences,gemini_iq70_speckle_exposure,gemini_iq85_speckle_exposure,IQ70 Program Time,IQ85 Program Time,Observable
str14,str28,str6,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,int64,int64,int64,float64,float64,float64,float64,bool
TIC 391620600,Kostov 2022 arXiv:2202.05790,QuadEB,71.6403891258743,44.7536621987808,-7.75529,2.23094,2.07298,12.772,5949.71,False,True,True,True,False,True,True,False,9,7,14,448.0,896.0,886.882656,1418.0637120000001,True
TIC 63459761,Kostov 2022 arXiv:2202.05790,QuadEB,308.525065221094,41.1358690935082,-1.85814,-4.47149,0.469204,13.277,3960.0,False,True,True,True,False,False,False,False,10,7,14,448.0,896.0,886.882656,1418.0637120000001,True
TIC 278352276,Kostov 2022 arXiv:2202.05790,QuadEB,307.503640407431,48.6070563353721,1.85387,6.57664,1.36265,10.387,7156.0,False,True,True,True,False,True,True,False,5,5,10,320.0,640.0,735.1166400000001,1114.53168,True
TIC 264402353,Kostov 2022 arXiv:2202.05790,QuadEB,323.090421291026,78.695150742343,2.19147,-0.747179,0.963037,12.46,6358.0,False,True,True,True,False,True,True,False,9,7,14,448.0,896.0,886.882656,1418.0637120000001,True
TIC 454140642,Kostov 2022 arXiv:2202.05790,QuadEB,64.7734730427625,0.900042404712982,2.82736,-10.8466,2.71215,10.409,6592.0,False,True,True,True,False,True,True,False,3,5,10,320.0,640.0,735.1166400000001,1114.53168,True
TIC 459959916,Kostov 2022 arXiv:2202.05790,QuadEB,71.4896654350755,4.82961853921165,2.91745,5.87658,0.94933,13.323,6197.0,False,True,True,True,False,True,False,False,9,7,14,448.0,896.0,886.882656,1418.0637120000001,True
TIC 204698586,Kostov 2022 arXiv:2202.05790,QuadEB,185.551846252,-24.2248455504102,1.15779,5.32292,2.29163,11.506,6392.0,False,True,True,True,False,False,True,False,6,5,10,320.0,640.0,735.1166400000001,1114.53168,True
TIC 271186951,Kostov 2022 arXiv:2202.05790,QuadEB,132.355841821942,-46.8680404812717,-4.99193,5.06608,0.455852,12.049,9933.0,False,True,True,True,False,False,False,False,1,7,14,448.0,896.0,886.882656,1418.0637120000001,True
